## Import the required libraries

In [5]:
from surprise import Reader, Dataset, SVDpp

In [6]:
from sklearn.model_selection import GridSearchCV

In [7]:
from surprise.model_selection import cross_validate

## Connect to Postgresql to integrate payment history

In [8]:
import psycopg2
conn = psycopg2.connect(dbname='postgres', user='postgres', password='postgres123', host='localhost')

## Function to get customer transaction data

In [10]:
def get_customer_data(customer_id):
    cursor = conn.cursor()
    cursor.execute(f"""
        SELECT payment_method, success, cashback_amount
        FROM payment_history
        WHERE customer_id = {customer_id}
    """)
    data = cursor.fetchall()
    cursor.close()
    for row in data:
        yield (row[0], 'dummy', row[1], row[2])  
    return

## Load Data from Database

In [17]:
reader = Reader(rating_scale=(0, 1))  
data = Dataset.from_reader(reader,get_customer_data(target_customer_id)
## since the database is currently empty we are getting this error

SyntaxError: incomplete input (3642395857.py, line 3)

## Training the model 

In [18]:
# We will b using the Singular Vqalue Decomposition algo in our model
algo = SVD()
trainset = data.build_full_trainset()
algo.fit(trainset)

NameError: name 'data' is not defined

## Function to recommend the best payment method

In [20]:
def recommend_payment(customer_id, num_recommendations=5):
    predictions = algo.predict(customer_id, None, rating_scale=(0, 1))
    top_recs = sorted(predictions, key=lambda x: x.est, reverse=True)[:num_recommendations]
    return [pred.sid for pred in top_recs]